# Бинарная классификация с использованием набора данных об оттоке банковских средств

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Путь к файлу
file_path = "Churn_Modelling.csv"

# Загрузка датасета
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "shubhammeshram579/bank-customer-churn-prediction",
  file_path,
)

# Выведем первые строки
df.head()

100%|██████████| 669k/669k [00:00<00:00, 11.3MB/s]


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42.0,2,0.00,1,1.0,1.0,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41.0,1,83807.86,1,0.0,1.0,112542.58,0
2,3,15619304,Onio,502,France,Female,42.0,8,159660.80,3,1.0,0.0,113931.57,1
3,4,15701354,Boni,699,France,Female,39.0,1,0.00,2,0.0,0.0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43.0,2,125510.82,1,NaN,1.0,79084.10,0


In [ ]:
df.shape

(10002, 14)

In [ ]:
# Удалим лишнюю колонку с номерами строк
df.drop(['RowNumber'], axis=1, inplace=True)

In [ ]:
# Выведем информацию о датафрейме
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10002 entries, 0 to 10001
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       10002 non-null  int64  
 1   Surname          10002 non-null  object 
 2   CreditScore      10002 non-null  int64  
 3   Geography        10001 non-null  object 
 4   Gender           10002 non-null  object 
 5   Age              10001 non-null  float64
 6   Tenure           10002 non-null  int64  
 7   Balance          10002 non-null  float64
 8   NumOfProducts    10002 non-null  int64  
 9   HasCrCard        10001 non-null  float64
 10  IsActiveMember   10001 non-null  float64
 11  EstimatedSalary  10002 non-null  float64
 12  Exited           10002 non-null  int64  
dtypes: float64(5), int64(5), object(3)
memory usage: 1016.0+ KB


Датасет содержит следующие колонки:
* `CustomerId` (int64) - Уникальный идентификатор клиента
* `Surname` (object) - Фамилия клиента
* `CreditScore` (int64) - Кредитный рейтинг клиента
* `Geography` (object) - Гражданство клиента (France, Spain or Germany)
* `Gender` (object) - Пол клиента (Male or Female)
* `Age` (float64) - Возраст клиента
* `Tenure` (int64) - Длительность удержания клиента в банке (в годах)
* `Balance` (float64) - Баланс клиента
* `NumOfProducts` (int64) - Количество банковских продуктов у клиента
* `HasCrCard` (float64) - Наличие у клиента кредитной карты (1 = yes, 0 = no)
* `IsActiveMember` (float64) - Является ли клиент активным (1 = yes, 0 = no)
* `EstimatedSalary` (float64) - Предполагаемая зарплата клиента
* `Exited` (int64) - Целевая переменная, Whether the customer has churned (1 = yes, 0 = no)

In [ ]:
# Удалим неинформативные колонки с идентификаторами клиентов и их фамилиями
df.drop(['CustomerId', 'Surname'], axis=1, inplace=True)

# Обработка пропущенных значений

In [ ]:
# Количество пропущенных значений
df.isnull().sum()

,0
CreditScore,0
Geography,1
Gender,0
Age,1
Tenure,0
Balance,0
NumOfProducts,0
HasCrCard,1
IsActiveMember,1
EstimatedSalary,0


In [ ]:
# Доля пропущенных значений
df.isnull().sum() / df.shape[0]

,0
CreditScore,0.0000
Geography,0.0001
Gender,0.0000
Age,0.0001
Tenure,0.0000
Balance,0.0000
NumOfProducts,0.0000
HasCrCard,0.0001
IsActiveMember,0.0001
EstimatedSalary,0.0000


In [ ]:
# Выведем строки с пропущенными значениями
df[df['Age'].isnull() | df['Geography'].isnull() | df['HasCrCard'].isnull() | df['IsActiveMember'].isnull()]

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
4,850,Spain,Female,43.0,2,125510.82,1,NaN,1.0,79084.10,0
6,822,NaN,Male,50.0,7,0.00,2,1.0,1.0,10062.80,0
8,501,France,Male,44.0,4,142051.07,2,0.0,NaN,74940.50,0
9,684,France,Male,NaN,2,134603.88,1,1.0,1.0,71725.73,0


Пропуски содержатся в 4 колонках:


1.   `Age` - заполним пропуск медианным значением возраста
2.   `Geography` - заполним самым частовстречаемым значением
3.   Строки с пропусками в колонках `HasCrCard` и `IsActiveMember` удалим (их всего 2)



In [ ]:
# Заполненим пропуск в Geography
geography_mode = df['Geography'].mode()[0]
df['Geography'].fillna(geography_mode, inplace=True)

# Заполненим пропуск в Age
age_median = df['Age'].median()
df['Age'].fillna(df['Age'].median(), inplace=True)

# Остальные пропущенные значения удалим
df.dropna(inplace=True, ignore_index=True)

# Удалим дубликаты
df.drop_duplicates(inplace=True, ignore_index=True)

# Проверим, что не осталось пропущенных значений
print("Количество пропущенных значений:", df.isnull().values.sum())

Количество пропущенных значений: 0


# Анализ основных статистик

In [ ]:
df.describe()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,9998.000000,9998.000000,9998.000000,9998.000000,9998.000000,9998.000000,9998.000000,9998.000000,9998.000000
mean,650.523805,38.921987,5.013203,76474.427985,1.530206,0.705541,0.515003,100094.856392,0.203741
std,96.630805,10.487975,2.892289,62398.273540,0.581669,0.455822,0.499800,57515.311385,0.402799
min,350.000000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000,11.580000,0.000000
25%,584.000000,32.000000,3.000000,0.000000,1.000000,0.000000,0.000000,50983.750000,0.000000
50%,652.000000,37.000000,5.000000,97173.290000,1.000000,1.000000,1.000000,100218.210000,0.000000
75%,717.750000,44.000000,7.000000,127641.417500,2.000000,1.000000,1.000000,149395.882500,0.000000
max,850.000000,92.000000,10.000000,250898.090000,4.000000,1.000000,1.000000,199992.480000,1.000000


Основываясь на статистиках:
* Возраст клиентов варьируется от 18 до 92 лет
* Удержание от 0 до 10 лет
* Баланс от 0 до 250 тыс
* Количество продуктов от 1 до 4
* Предполагаемая зарплата от 11 до 200 тыс

Все значения выглядят правдоподобными

In [ ]:
df['Exited'].value_counts(normalize=True)

,proportion
Exited,
0,0.796259
1,0.203741


Классы несбалансированы, 80% относятся к классу exited

# Разделение признаков

In [ ]:
# Выделим числовые признаки
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance',
           'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']

# Выделим категориальные признаки
categorical = ['Geography', 	'Gender']

In [ ]:
print('В колонке Geography встречается', len(df['Geography'].unique()), 'уникальных значения:', df['Geography'].unique())

В колонке Geography встречается 3 уникальных значения: ['France' 'Spain' 'Germany']


In [ ]:
print('В колонке Gender встречается', len(df['Gender'].unique()), 'уникальных значения:', df['Gender'].unique())

В колонке Gender встречается 2 уникальных значения: ['Female' 'Male']


# Кодирование категориальных признаков

Для признака `Geography` будем использовать One-Hot Encoding

In [ ]:
df_encoded = pd.get_dummies(df, columns=['Geography'])
df_encoded

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,Female,42.0,2,0.00,1,1.0,1.0,101348.88,1,True,False,False
1,608,Female,41.0,1,83807.86,1,0.0,1.0,112542.58,0,False,False,True
2,502,Female,42.0,8,159660.80,3,1.0,0.0,113931.57,1,True,False,False
3,699,Female,39.0,1,0.00,2,0.0,0.0,93826.63,0,True,False,False
4,645,Male,44.0,8,113755.78,2,1.0,0.0,149756.71,1,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9993,771,Male,39.0,5,0.00,2,1.0,0.0,96270.64,0,True,False,False
9994,516,Male,35.0,10,57369.61,1,1.0,1.0,101699.77,0,True,False,False
9995,709,Female,36.0,7,0.00,1,0.0,1.0,42085.58,1,True,False,False
9996,772,Male,42.0,3,75075.31,2,1.0,0.0,92888.52,1,False,True,False


В результате появилось 3 новых признака `Geography_France`,	`Geography_Germany`,	`Geography_Spain`

Для признака `Gender` будем использовать бинарное кодирование. Сопоставим Male: 0, Female: 1

In [ ]:
df_encoded['Gender'] = df_encoded['Gender'].map({'Male': 0, 'Female': 1})
df_encoded

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,1,42.0,2,0.00,1,1.0,1.0,101348.88,1,True,False,False
1,608,1,41.0,1,83807.86,1,0.0,1.0,112542.58,0,False,False,True
2,502,1,42.0,8,159660.80,3,1.0,0.0,113931.57,1,True,False,False
3,699,1,39.0,1,0.00,2,0.0,0.0,93826.63,0,True,False,False
4,645,0,44.0,8,113755.78,2,1.0,0.0,149756.71,1,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9993,771,0,39.0,5,0.00,2,1.0,0.0,96270.64,0,True,False,False
9994,516,0,35.0,10,57369.61,1,1.0,1.0,101699.77,0,True,False,False
9995,709,1,36.0,7,0.00,1,0.0,1.0,42085.58,1,True,False,False
9996,772,0,42.0,3,75075.31,2,1.0,0.0,92888.52,1,False,True,False


# Нормализация

Выделим признаки, которые необходимо нормализовать, это те признаки, которые принимают непрерывные значения

In [ ]:
continuous_columns = ['CreditScore','Age','Tenure','Balance',
                      'NumOfProducts','EstimatedSalary']

scaler = MinMaxScaler()
df_encoded[continuous_columns] = scaler.fit_transform(df_encoded[continuous_columns])

df_encoded

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,0.538,1,0.324324,0.2,0.000000,0.000000,1.0,1.0,0.506735,1,True,False,False
1,0.516,1,0.310811,0.1,0.334031,0.000000,0.0,1.0,0.562709,0,False,False,True
2,0.304,1,0.324324,0.8,0.636357,0.666667,1.0,0.0,0.569654,1,True,False,False
3,0.698,1,0.283784,0.1,0.000000,0.333333,0.0,0.0,0.469120,0,True,False,False
4,0.590,0,0.351351,0.8,0.453394,0.333333,1.0,0.0,0.748797,1,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9993,0.842,0,0.283784,0.5,0.000000,0.333333,1.0,0.0,0.481341,0,True,False,False
9994,0.332,0,0.229730,1.0,0.228657,0.000000,1.0,1.0,0.508490,0,True,False,False
9995,0.718,1,0.243243,0.7,0.000000,0.000000,0.0,1.0,0.210390,1,True,False,False
9996,0.844,0,0.324324,0.3,0.299226,0.333333,1.0,0.0,0.464429,1,False,True,False
